In [346]:
import requests
import os
import re
import json
import datetime
import time
import pandas as pd
import numpy as np
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

import urllib
from urllib import parse,request
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument("--headless")

import jieba
import re

In [302]:
!pip install jieba

    100% |████████████████████████████████| 7.3MB 672kB/s ta 0:00:011
  Running setup.py bdist_wheel for jieba ... done
  Stored in directory: /Users/gaohuaxuan/Library/Caches/pip/wheels/c9/c7/63/a9ec0322ccc7c365fd51e475942a82395807186e94f0522243
Successfully built jieba
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Scrapper

In [258]:
def get_content_by_page(page):
    # page = 1~100     
    setA = ['x','y','z', 0,1,2,3,4,5]
    setB = ['M','N','O']
    setC = ['A','Q','g','w']
    url = 'http://data.eastmoney.com/report/hyyb.html#dHA9MCZjZz0wJmR0PTQmcGFnZT0'
    if(page<10):
        url += str(setA[(page-1)%10])
    elif(page==100):
        url += "xMDA="
    else:
        url += str(setA[((page//10)-1)%10])
        url += setB[(page%10)//4] + setC[(page%10)%4] + "=="
#     print(url)
    driver.get(url)
    data = driver.page_source
    soup = BeautifulSoup(data, 'html5lib')
    return soup

In [248]:
def filter_data(soup):
    a_tags = soup.find_all('a', href=True)
    titles = []
    reports = []
    indus = []
    for a in a_tags:
        title = a.get('title')
        href = a.get('href')
        if(title!=None):
            titles.append(title)
            reports.append(href)
        if("quote.eastmoney.com" in href):
            indus.append(a.contents)
    return titles[:50], reports[:50], indus[3:]

In [249]:
def get_txt(href):
    url = "http://data.eastmoney.com"+href
    opener = request.urlopen(url)
#     print(url)
    content = opener.read()
    soup = BeautifulSoup(content, 'html.parser')
    p_contents = soup.find_all('p')
    text = ""
    for p in p_contents:
        if(p.contents!=[]):
            text += p.contents[0]
    return text

In [255]:
for i in range(1,101):
    # loop pages
    driver = webdriver.Chrome(options=chrome_options)
    print("page: "+str(i))
    soup = get_content_by_page(i)
    title, report, indus = filter_data(soup)
    if(len(title)==len(report) and len(title)==len(indus)):
        # print("length check passed")
        # loop 50 articles
        for j, href in enumerate(report):
            try:
                text = get_txt(href)
            except:
                print(title[j] + "url not found: http://data.eastmoney.com"+href)
                continue
            row = pd.DataFrame(data={'industry':indus[j], 'title':title[j], 'text':text})
            row = row[['industry','title','text']]
            row.to_csv('result_'+str(i)+'.csv', mode='a', header=False, index=False)
    else:
        print("length not equal!")
        print(len(title))
        print(len(report))
        print(len(indus))
        break
    driver.close()

page: 1
page: 2
page: 3
page: 4
2019年上海地区升学体系及升学政策研究url not found: http://data.eastmoney.com/report/20190514/hy,APPJ6Kps7gbIIndustry.html
page: 5
page: 6
page: 7
page: 8
page: 9
page: 10
page: 11
page: 12
page: 13
page: 14
page: 15
page: 16
page: 17
page: 18
page: 19
page: 20
page: 21
page: 22
page: 23
page: 24
page: 25
page: 26
page: 27
page: 28
page: 29
page: 30
page: 31
page: 32
page: 33
page: 34
page: 35
page: 36
page: 37
page: 38
page: 39
房地产行业深度报告：高股息率地产股值得关注url not found: http://data.eastmoney.com/report/20190425/hy,APPJ5yxxTlxRIndustry.html
page: 40
page: 41
page: 42
page: 43
page: 44
page: 45
page: 46
page: 47
page: 48
page: 49
page: 50
page: 51
page: 52
page: 53
page: 54
page: 55
page: 56
page: 57
page: 58
page: 59
page: 60
page: 61
page: 62
page: 63
page: 64
page: 65
page: 66
page: 67
page: 68
page: 69
page: 70
page: 71
page: 72
page: 73
page: 74
page: 75
page: 76
教育行业跨市场周报（第68期）：职业教育政策红利‚重点关注中公开元url not found: http://data.eastmoney.com/report/20190408/hy,APPJ5uwAXtinIndustr

IndexError: list index out of range

# Preprocessing

In [380]:
from keras.preprocessing.text import Tokenizer

In [ ]:
res_df = pd.DataFrame()
for i in range(1,101):
    df = pd.read_csv("result_"+str(i)+".csv", header=None)
    res_df = pd.concat([res_df, df])
res_df = res_df.rename(columns={0:'industry', 1:'title', 2:'text'})

In [274]:
indus_set = set(res_df['industry'].values)

In [389]:
i = 0
label_map = {}
for cat in indus_set:
    label_map[cat] = i
    i += 1

In [451]:
labels = []
for indus in res_df['industry']:
    labels.append(label_map[indus])

In [452]:
labels = np.array(labels)

In [354]:
vocab = set() # vocab list
words = [] # train data
for i in range(len(res_df)):
    text = res_df['text'].iloc[i]
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）：-；]", "", text)
    words.append(list(jieba.cut(text)))
#     vocab = vocab | set(list(jieba.cut(res_df['text'].iloc[i])))

In [381]:
t = Tokenizer()
t.fit_on_texts(words)

In [383]:
encoded_docs = t.texts_to_matrix(words, mode='count')
print(encoded_docs)

[[ 0. 19.  5. ...  0.  0.  0.]
 [ 0. 12. 17. ...  0.  0.  0.]
 [ 0.  3. 16. ...  0.  0.  0.]
 ...
 [ 0. 34.  7. ...  0.  0.  0.]
 [ 0. 11.  6. ...  0.  0.  0.]
 [ 0.  3.  4. ...  1.  1.  1.]]


In [384]:
encoded_docs.shape

(4995, 67825)

In [404]:
max_strlen 

4535

# Model

In [458]:
import keras
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense, Dropout, Embedding
from sklearn.model_selection import StratifiedKFold
from keras import backend as K

In [459]:
# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [460]:
def build_model():
    model = Sequential()

    # model.add(
    #     Embedding(input_dim=len(vocab),
    #               input_length = encoded_docs.shape[1],
    #               output_dim=100,
    #               trainable=True,
    #               mask_zero=True))

    # Recurrent layer
    # model.add(LSTM(64, return_sequences=False,
    #                dropout=0.1, recurrent_dropout=0.1))

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation='softmax'))
    
    model.compile(
        optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[f1])
    
    return model

In [461]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=2019)
cvscores = []
X = encoded_docs
Y = labels
kfold_split = kfold.split(X, Y)

In [463]:
for train, test in kfold_split:
  # create model
    model = build_model()
    model.fit(X[train], Y[train], epochs=5, batch_size=512, verbose=1)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=1)
    print("%s: %.4f" % (model.metrics_names[1], scores[1]))
    cvscores.append(scores[1])

Epoch 1/5
4487/4487 [==============================] - 16s 3ms/step - loss: 2.5921 - f1: 28.0763
Epoch 2/5
4487/4487 [==============================] - 11s 3ms/step - loss: 0.8309 - f1: 6.2196
Epoch 3/5
4487/4487 [==============================] - 11s 2ms/step - loss: 0.4428 - f1: 3.0210
Epoch 4/5
4487/4487 [==============================] - 14s 3ms/step - loss: 0.2946 - f1: 2.2643
Epoch 5/5
508/508 [==============================] - 3s 5ms/step
f1: 2.1372
Epoch 1/5
4487/4487 [==============================] - 19s 4ms/step - loss: 2.5497 - f1: 27.8749
Epoch 2/5
4487/4487 [==============================] - 14s 3ms/step - loss: 0.8096 - f1: 5.9624
Epoch 3/5
4487/4487 [==============================] - 14s 3ms/step - loss: 0.4366 - f1: 2.8954
Epoch 4/5
4487/4487 [==============================] - 14s 3ms/step - loss: 0.2939 - f1: 2.2962
Epoch 5/5
508/508 [==============================] - 3s 5ms/step
f1: 2.3474
Epoch 1/5
4497/4497 [==============================] - 21s 5ms/step - loss: 2.

NameError: name 'numpy' is not defined

In [466]:
print("%.2f (+/- %.2f)" % (np.mean([cvscores]), np.std([cvscores])))

2.03 (+/- 0.17)
